<a href="https://colab.research.google.com/github/swaty369/Incident-Management-System/blob/final_files/statistical_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns',None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Importing data

In [ ]:
# importing data which is already scaled and encoded and transformations are done i,e cleaned data
data=pd.read_csv("/content/drive/My Drive/GL Capstone/Incident_data_before_encode.csv", index_col= 'Unnamed: 0')
data.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,opened_by,sys_created_by,sys_updated_by,contact_type,location,category,subcategory,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,resolved_by,predicted_closed_days,days_took_to_sys_created,days_took_to_sys_updated,days_took_to_resolve,days_took_to_close,opened_month,opened_day,opened_hour,opened_minute,sys_updated_month,sys_updated_day,sys_updated_hour,sys_updated_minute,closed_month,closed_day,closed_hour,closed_minute,sys_created_month,sys_created_day,sys_created_hour,sys_created_minute,resolved_month,resolved_day,resolved_hour,resolved_minute
0,INC0000045,New,True,0.0,0,0.000000,True,Opened by 8,Created by 6,Updated by 21,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,Resolved by 149,5.447222,0.004861,0.000000,0.420833,5.021528,2,29,1,16,2,29,1,23,3,5,12,0,2,29,1,23,2,29,11,29
1,INC0000047,New,True,0.0,0,0.000000,True,Opened by 397,Created by 171,Updated by 746,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify,Resolved by 81,6.222222,0.011806,0.000000,1.204861,5.005556,2,29,4,40,2,29,4,57,3,6,10,0,2,29,4,57,3,1,9,52
2,INC0000047,Active,True,1.0,0,1.000000,True,Opened by 397,Created by 171,Updated by 21,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 24,Resolver 31,True,False,Do Not Notify,Resolved by 81,6.222222,0.011806,0.022917,1.181944,5.005556,2,29,4,40,2,29,5,30,3,6,10,0,2,29,4,57,3,1,9,52
3,INC0000047,Active,True,1.0,0,1.414214,True,Opened by 397,Created by 171,Updated by 21,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 24,Resolver 31,True,False,Do Not Notify,Resolved by 81,6.222222,0.011806,0.025000,1.179861,5.005556,2,29,4,40,2,29,5,33,3,6,10,0,2,29,4,57,3,1,9,52
4,INC0000047,Active,True,1.0,0,1.732051,True,Opened by 397,Created by 171,Updated by 804,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 24,Resolver 31,True,False,Do Not Notify,Resolved by 81,6.222222,0.011806,0.273611,0.931250,5.005556,2,29,4,40,2,29,11,31,3,6,10,0,2,29,4,57,3,1,9,52


In [ ]:
feat_num = data.select_dtypes(np.number).columns
feat_cat = data.select_dtypes(['object', 'bool']).columns
feat_cat

Index(['number', 'incident_state', 'active', 'made_sla', 'opened_by',
       'sys_created_by', 'sys_updated_by', 'contact_type', 'location',
       'category', 'subcategory', 'u_symptom', 'impact', 'urgency', 'priority',
       'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'resolved_by'],
      dtype='object')

In [ ]:
data.columns

Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'opened_by',
       'sys_created_by', 'sys_updated_by', 'contact_type', 'location',
       'category', 'subcategory', 'u_symptom', 'impact', 'urgency', 'priority',
       'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'resolved_by',
       'predicted_closed_days', 'days_took_to_sys_created',
       'days_took_to_sys_updated', 'days_took_to_resolve',
       'days_took_to_close', 'opened_month', 'opened_day', 'opened_hour',
       'opened_minute', 'sys_updated_month', 'sys_updated_day',
       'sys_updated_hour', 'sys_updated_minute', 'closed_month', 'closed_day',
       'closed_hour', 'closed_minute', 'sys_created_month', 'sys_created_day',
       'sys_created_hour', 'sys_created_minute', 'resolved_month',
       'resolved_day', 'resolved_hour', 'resolved_minute'],
      dtype='object')

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
insignificant_features =[]
significant_features =[]

for feat in feat_cat[1:]:
    formula = ('predicted_closed_days ~ '+feat)
    
    model = ols(formula, data=data).fit()
    aov_table = anova_lm(model, typ=2)
    # print(formula)
    # print(aov_table)
    # print('****************************')
    
    pvalue = pd.DataFrame(aov_table)['PR(>F)'][0]

    # print(f"\nPvalue w.r.t 0.05 signinficance {np.round(pvalue,3)}")
    if pvalue > 0.05:
        insignificant_features.append(feat)
        print(f"{feat} is insignificant with pvalue {pvalue} ")
        
        
    else:
        significant_features.append(feat)
        print(f"{feat} is significant with pvalue {pvalue} ")

incident_state is significant with pvalue 0.0 
active is significant with pvalue 0.0 
made_sla is significant with pvalue 1.3983943920387197e-84 
opened_by is significant with pvalue 0.0 
sys_created_by is significant with pvalue 0.0 
sys_updated_by is significant with pvalue 0.0 
contact_type is significant with pvalue 1.4287959173907828e-10 
location is significant with pvalue 1.2893949475202491e-275 
category is significant with pvalue 0.0 
subcategory is significant with pvalue 0.0 
u_symptom is significant with pvalue 0.0 
impact is significant with pvalue 6.986105334828315e-40 
urgency is significant with pvalue 2.1813680254777426e-33 
priority is significant with pvalue 4.0385794784325824e-54 
assignment_group is significant with pvalue 0.0 
assigned_to is significant with pvalue 0.0 
knowledge is significant with pvalue 1.599053907395703e-244 
u_priority_confirmation is significant with pvalue 2.5580252685780908e-45 
notify is insignificant with pvalue 0.17273700052643043 
reso

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
insignificant_features =[]
significant_features =[]

for feat in feat_num:
    formula = ('predicted_closed_days ~ '+feat)
    
    model = ols(formula, data=data).fit()
    aov_table = anova_lm(model, typ=2)
    # print(formula)
    # print(aov_table)
    # print('****************************')
    
    pvalue = pd.DataFrame(aov_table)['PR(>F)'][0]

    # print(f"\nPvalue w.r.t 0.05 signinficance {np.round(pvalue,3)}")
    if pvalue > 0.05:
        insignificant_features.append(feat)
        print(f"{feat} is insignificant with pvalue {pvalue} ")
        
        
    else:
        significant_features.append(feat)
        print(f"{feat} is significant with pvalue {pvalue} ")

reassignment_count is significant with pvalue 0.0 
reopen_count is significant with pvalue 3.204222418521365e-42 
sys_mod_count is significant with pvalue 0.0 
predicted_closed_days is significant with pvalue 0.0 
days_took_to_sys_created is significant with pvalue 9.138425726103324e-27 
days_took_to_sys_updated is significant with pvalue 0.0 
days_took_to_resolve is significant with pvalue 0.0 
days_took_to_close is significant with pvalue 1.0678494278620087e-248 
opened_month is significant with pvalue 2.4958448603866497e-74 
opened_day is significant with pvalue 6.06062215027349e-77 
opened_hour is significant with pvalue 2.1635090374606924e-14 
opened_minute is significant with pvalue 0.0010842992525125607 
sys_updated_month is significant with pvalue 0.0 
sys_updated_day is significant with pvalue 3.080657294030578e-31 
sys_updated_hour is significant with pvalue 8.120310983535843e-17 
sys_updated_minute is significant with pvalue 0.0038047747006111784 
closed_month is significant

In [ ]:
# importing data which is already scaled and encoded and transformations are done i,e cleaned data
data=pd.read_csv("/content/drive/My Drive/GL Capstone/incident_management_cleaned.csv.csv")
data.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,resolved_at,closed_at,predicted_closed_days,opened_month,opened_day,opened_hour,opened_minute,sys_updated_month,sys_updated_day,sys_updated_hour,sys_updated_minute
0,INC0000045,New,True,0.0,0,0.000000,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,2016-02-29 01:23:00,Updated by 21,2016-02-29 01:23:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00,5.447222,2,29,1,16,2,29,1,23
1,INC0000045,Resolved,True,0.0,0,1.414214,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,2016-02-29 01:23:00,Updated by 642,2016-02-29 08:53:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00,5.447222,2,29,1,16,2,29,8,53
2,INC0000045,Resolved,True,0.0,0,1.732051,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,2016-02-29 01:23:00,Updated by 804,2016-02-29 11:29:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00,5.447222,2,29,1,16,2,29,11,29
3,INC0000045,Closed,False,0.0,0,2.000000,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,2016-02-29 01:23:00,Updated by 908,2016-03-05 12:00:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,2016-02-29 11:29:00,2016-03-05 12:00:00,5.447222,2,29,1,16,3,5,12,0
4,INC0000047,New,True,0.0,0,0.000000,True,Caller 2403,Opened by 397,2016-02-29 04:40:00,Created by 171,2016-02-29 04:57:00,Updated by 746,2016-02-29 04:57:00,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify,code 5,Resolved by 81,2016-03-01 09:52:00,2016-03-06 10:00:00,6.222222,2,29,4,40,2,29,4,57


In [ ]:
# chi2 contingency test for categorical proportion difference
# filterring rows where data is closed
data.incident_state.unique()

data_closed = data[data.incident_state=='Closed']

In [ ]:
data_closed.columns

Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'impact', 'urgency', 'priority', 'assignment_group',
       'assigned_to', 'knowledge', 'u_priority_confirmation', 'notify',
       'closed_code', 'resolved_by', 'resolved_at', 'closed_at',
       'predicted_closed_days', 'opened_month', 'opened_day', 'opened_hour',
       'opened_minute', 'sys_updated_month', 'sys_updated_day',
       'sys_updated_hour', 'sys_updated_minute'],
      dtype='object')

In [ ]:
feat_other = ['contact_type', 'location', 'category', 'subcategory', 'u_symptom',
       'priority', 'knowledge', 'u_priority_confirmation',
       'reassignment_count', 'reopen_count', 'sys_mod_count']

In [ ]:
df = pd.crosstab(Loan.Loan_status, Loan.Profession)
from scipy.stats import chi2_contingency, chisquare

stat, pvalue, ddof, arr = chi2_contingency(df)

print(f"Pvalue: {pvalue}")

if pvalue>0.05:
    print("Null accepted : Defaulters does not depends on profession")
else:
    print("Null rejected : Defaulters depends on profession")

NameError: ignored

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import chi2_contingency, chisquare
insignificant_features =[]
significant_features =[]

for feat in data_closed.columns[1:]:
    df = pd.crosstab(data_closed['made_sla'], data_closed[feat])
    # display(df)
    stat, pvalue, ddof, arr = chi2_contingency(df)
    

    # print(f"\nPvalue w.r.t 0.05 signinficance {np.round(pvalue,3)}")
    if pvalue > 0.05:
        # insignificant_features.append(feat)
        print(f"{feat} is having same proportion , pvalue {pvalue} ")
        
        
    else:
        # significant_features.append(feat)
        print(f"{feat} is having different proportion , pvalue {pvalue} ")
        

incident_state is having same proportion , pvalue 1.0 
active is having same proportion , pvalue 1.0 
reassignment_count is having different proportion , pvalue 0.0 
reopen_count is having different proportion , pvalue 3.0093213985514036e-38 
sys_mod_count is having different proportion , pvalue 0.0 
made_sla is having different proportion , pvalue 0.0 
caller_id is having different proportion , pvalue 7.562919385968114e-77 
opened_by is having different proportion , pvalue 0.0 
opened_at is having different proportion , pvalue 3.3490750415695158e-15 
sys_created_by is having different proportion , pvalue 0.0 
sys_created_at is having different proportion , pvalue 2.2043268814275758e-15 
sys_updated_by is having different proportion , pvalue 0.0006899528266100729 
sys_updated_at is having different proportion , pvalue 0.0 
contact_type is having different proportion , pvalue 1.104297700366967e-06 
location is having different proportion , pvalue 1.3211866784352722e-83 
category is havi

In [ ]:
# models =  [('Linear Model',lr),('Random Forest Regressor', rfr),('XGB Regressor',xgb),
#             ('Gradient Boosing Regressor', gbr),('SGD Regressor',sgdr)]
# result = []
# model_Name =[]
# for i,j in models:
#     cv_results = cross_val_score(j, xtrain,ytrain, cv = 5)
#     model.append(j)
#     result.append(cv_results)
#     print(i,'Mean', np.mean(cv_results), 'Std',np.std(cv_results))
# fig = plt.figure()
# fig.suptitle('Machine Learning Model Comparison')
# ax = fig.add_subplot(111)
# plt.boxplot(result)
# ax.set_xticklabels(model)
# plt.show()
# below one is for kfold
# models =  [('Linear Model',lr),('Random Forest Regressor', rfr),('XGB Regressor',xgb),
#             ('Gradient Boosing Regressor', gbr),('SGD Regressor',sgdr)]
# result = []
# model_Name =[]
# for i,j in models:
#     kfold=KFold(n_splits=10,shuffle=True)  
#     kfoldcv_results = cross_val_score(j, xtrain,ytrain, cv = kfold)
#     model.append(j)
#     result.append(kfoldcv_results)
#     print(i,'Mean', np.mean(kfoldcv_results), 'Std',np.std(kfoldcv_results))